## Step # 01: Install necessary libraries
- Crewai[tools]: It is a framework for autonomous AI agents designed for Reasoning and Acting.
The [tools] part installs additional built-in tools that can be used by AI agents (e.g., web search, calculations, file handling).
- duckduckgo-search: A Python library for performing DuckDuckGo searches programmatically. Search for web pages, news, images, and videos.



In [ ]:
%pip install 'crewai[tools]'==0.28.8 
%pip duckduckgo-search==5.3.0 

## Step # 02: Import necessary libraries

In [2]:
import json                                                  #for handling JSON data
import os                                                    #for interacting with operating system(file paths, env-variables)
import re                                                    #for working with regular expressions(text pattern matching)
import requests                                              #for making HTTPS requests(sending or getting data from API's)
from datetime import datetime                                #for date and time
from crewai import Agent, Crew, Process, Task                #framework to create autonomous AI Agents
from IPython.display import Markdown                         #display Markdown-formatted text inside Jupyter Notebook output cells
from langchain.agents import load_tools                      #LangChain is a framework used to build applications powered by large language models (LLMs)
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI


/home/mahrukh/NLP/Agents/venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [3]:
#Function: to process a string containing multiple comma-separated entries enclosed in square brackets and return a cleaned list of those entries. 
def format_response(response: str) -> str:
    entries = re.split(r", (?=\[)", response) 
    return [entry.strip("[]") for entry in entries] 

## Step # 03: Set OPEN_API_KEY

In [ ]:
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_API_KEY>"

## Step # 04: Initializes a DuckDuckGo Search tool
The tool will fetch news-related search results with a specified number of results.

In [5]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=13)

## Step # 05: Data Source

In [6]:
response = search_tool.run("Earthquake")
format_response(response)

['snippet: Ndlula, Umngani and Khosi — scramble to encircle and shield the two 7-year-old calves, Zuli and Mkhaya, from any possible threats., title: Video shows elephants at San Diego Zoo Safari Park racing to protect their young during earthquake, link: https://www.msn.com/en-us/public-safety-and-emergencies/health-and-safety-alerts/elephants-at-san-diego-zoo-huddle-to-protect-calves-during-earthquake/ar-AA1CVa1c, date: 2025-04-15T21:09:00+00:00, source: The Associated Press',
 "snippet: A preliminary 6.0-magnitude earthquake struck near San Diego, according to the U.S. Geological Survey. The quake's epicenter was recorded 1.86 northeast of San Diego County, in Julian. It is not yet known if there are reports of injuries or damage., title: 5.2-magnitude earthquake strikes San Diego area, jolting Southern California near San Andreas system fault, link: https://www.cbsnews.com/losangeles/news/earthquake-strikes-near-san-diego/, date: 2025-04-15T07:40:00+00:00, source: CBS News",
 'snip

In [9]:
response = search_tool.run("Rainfall")
format_response(response)

["snippet: It may feel like it rained a lot last year - and it did - but not enough to satisfy L.A. County's critical shortage of rainfall, officials say. A news release issued Tuesday by Water for L.A. County indicates that the county captured nearly 12 billion gallons of stormwater over the past several months., title: L.A. County remains 'critically short' on rainfall despite recent storms, link: https://www.msn.com/en-us/weather/topstories/l-a-county-remains-critically-short-on-rainfall-despite-recent-storms/vi-AA1D2H2N, date: 2025-04-16T14:27:02+00:00, source: KTLA-TV Los Angeles",
 "snippet: MEMPHIS, Tenn. - Life-threatening flooding and dangerous severe weather pummeled large swaths of the nation's heartland again Saturday for the fourth day in a row, promising to leave some areas with heavy rainfall not seen in generations and triggering ..., title: Catastrophic rainfall triggers Flash Flood Emergencies, evacuations as relentless storms drag into fourth day, link: https://www.a

## Step # 06: Tools

In [10]:
human_tools = load_tools(["human"])

## Step # 07: Load GPT-3.5 Model

In [11]:
llm = ChatOpenAI(
    model = "gpt-3.5-turbo"
)

## Step # 08: Initialize Agents

In [12]:
#Senior Environmental Customer Communicator
customer_communicator = Agent (
    role="Senior environmental and natural events communicator",
    goal="Understand what natural events the customer is asking about, such as earthquake, rainfall, weather patterns, or environmental changes, and provide clear, accurate, and relevanr information",
    backstory="""You're a highly experienced specialist in communicating about natural events including earthquakes, rainfall, weather, and environmental conditions. 
    You help users by answering their questions and providing helpful explanations, facts and insights on these topics.""",
    verbose=True, 
    allow_delegation=False,     #handles things directly without delegating
    llm=llm,
    max_iter=5,
    memory=True,                #whether this agent remembers context/history
    tools=human_tools,          #tools available to this agent(search tools, APIs etc)
    )

#Environmental Events Analyst
analyst= Agent (
    role="Environmental Events Analyst",
    goal="""Create a short, clear summary of recent natural events and environmental news.
    Specify the current trend - increasing, decreasing, or stable - for key factors like earthquake frequency, rainfall levels, temperature changes, and other natural 
    phenomena. For each requested event type, specify a risk level between 0 and 100, where 0 indicates no risk or impact, and 100 represents extreme danger or urgency""",
    backstory="""You're widely accepted as one of the most reliable analysts in the field of natural events and environmental science.  
    You have tracked global environmental patterns for over a decade and your trend analysis is highly regarded for its accuracy.

    You are also deeply knowledgeable about climate science, disaster management, and human-environment interactions.
    You understand complex environmental systems and integrate multiple disciplines — such as meteorology, geology, and environmental psychology — when analyzing events.

    You assess news and reports critically, taking into account their sources and reliability.
    You always present concise and easy-to-understand summaries that explain complex environmental and natural event topics in simple, clear terms for the public and decision-makers.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=10,
    memory=True,
    tools=[search_tool],    # uses search tools or APIs for latest data
    )

#Environmental Newsletter Writer
writer = Agent (
    role="Environmental Newsletter Writer",
    goal="""Write an insightful, compelling, and informative 3-paragraph-long newsletter based on the latest environmental events and natural phenomena report.""",
    backstory="""You're deeply familiar with natural events, environmental science, and climate-related topics. 
    You understand complex concepts like earthquake science, rainfall patterns, climate trends, and their real-world impacts — and have a gift for transforming this technical information into engaging, clear, and easy-to-read articles for a general audience.

    You're a master of communication, using narratives and structure to keep readers engaged.
    Your writing focuses on clarity, conciseness, and a pleasant flow that makes technical content approachable.

    Some of your writing techniques include:

    - Start with a 3-bullet executive summary highlighting the most important points
    - Use bullet points and subheadings to make content skimmable and easy to follow
    - Use real-world examples and analogies to explain complex environmental ideas
    - Craft compelling introductions and conclusions to make your writing memorable and actionable

    Your writing style turns even the driest scientific data into engaging and interesting newsletters for readers of all backgrounds. """,
    verbose=True,
    allow_delegation=True,    # can delegate tasks to other agents if needed
    llm=llm,
    max_iter=10,
    memory=True,)


## Step # 09: Initialize Tasks for Agents

In [13]:
#Task for Senior Environmental Customer Communicator Agent
get_natural_event_topics = Task (
    description=f"""Ask the user which natural events or environmental topics they're interested in (like earthquake, rainfall, temperature trends, storms, or other climate events).""",
    expected_output="""Create a list of the natural events or environmental topics that the user wants information about.
    Example: 
    events = 'earthquake, rainfall, storm'
    """,
    agent=customer_communicator,
)

#Task for Environmental Events Analyst Agent
get_event_news_and_data = Task(
    description = f"""Take the list of natural events and always include 'earthquake' and 'rainfall' to it(if not requested)
    Use the search tool to gather news, updates, and scientific data for each topic individually. 
    The current date is {datetime.now()}.
    Summarize the information into a clear, helpful report.""",
    expected_output="""A summary of the current environmental situation and a one-sentence summary for
    each requested natural event or topic. Include a risk or impact score between 0 and 100 (0 being no risk/impact, 100 being severe risk/impact). Use this format:
    <EVENT>
    <SUMMARY BASED ON NEWS AND DATA>
    <CURRENT TREND OR PATTERN>
    <RISK/IMPACT SCORE>
    """,
    agent=analyst,
    context=[get_natural_event_topics],
)

#Task for Environmental Newsletter Writer Agent
write_environment_newsletter = Task(
    description = f"""
    Use the environmental events report to create a daily newsletter that highlights the most important points.
    Focus on the event summaries, current trends, and risk/impact scores. Discuss near-term and future considerations.
    The newsletter should be accessible to a wide audience and rely on clear, factual, and evidence-based information.""",
    expected_output = """An eloquent 3-paragraph newsletter formatted as markdown in an easy-to-read manner. It should contain: 
    - 3-bullet executive summary 
    - Introduction — set the overall context and hook the reader's interest
    - Main section — detailed analysis including news summaries and risk/impact scores
    - Summary — key facts and a future prediction: increasing, descreasing, or stable trend               
    """,
    agent=writer,
    context=[get_event_news_and_data],    
)

## Step # 10: Initialize Crew

In [14]:
crew = Crew (
    agents=[customer_communicator,analyst,writer],
    tasks=[get_event_news_and_data,get_natural_event_topics, write_environment_newsletter],
    verbose=2,                  #This controls how much output detail you get during execution: 0 = no output,1 = basic logs, 2 = detailed logs (which agents are doing what, task results, etc.) — good for debugging and learning.
    process=Process.hierarchical,  #how to coordinate the tasks: hierarchical, which is often more organized for multi-step workflows.
    full_output=True,
    share_crew=False,
    manager_llm=llm,               #LLM (Large Language Model) instance acting as the task manager for this crew, making decisions about task assignments and flow in hierarchical mode
    max_iter=15,
)

## Step # 11: Display output

In [ ]:
results = crew.kickoff()

In [ ]:
list(results.keys())

In [ ]:
len(results["tasks_outputs"])

In [ ]:
Markdown(results["tasks_outputs"][1].exported_output)

In [ ]:
Markdown(results["final_output"])